In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

In [6]:
# today = today - timedelta(days=1)

In [9]:


homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

# file_name = f"data/rankings_worksheet_{(today-timedelta(days=1)).strftime('%Y-%m-%d')}.xlsx"
file_name = f"data/rankings_worksheet_{(today).strftime('%Y-%m-%d')}.xlsx"
rankings = pd.read_excel(file_name)

last_season = pd.read_csv('data/2022-23.csv')

In [10]:
single_games = all_df.sort_values('fantasyPoints', ascending=False)

goaltenders_single = single_games.loc[single_games['position'] == 'G']
skaters_single = single_games.loc[single_games['position'] != 'G']
defense_single = skaters_single.loc[skaters_single['position'] == 'D']
forwards_single = skaters_single.loc[skaters_single['position'] != 'D']

In [11]:
defense_single.columns.to_list()

['playerId',
 'sweaterNumber',
 'name',
 'position',
 'goals',
 'assists',
 'points',
 'plusMinus',
 'pim',
 'hits',
 'blockedShots',
 'powerPlayGoals',
 'powerPlayPoints',
 'shorthandedGoals',
 'shPoints',
 'shots',
 'faceoffs',
 'toi',
 'powerPlayToi',
 'shorthandedToi',
 'team',
 'opponent',
 'secondaryPosition',
 'tertiaryPosition',
 'gameDate',
 'gameTime',
 'gameId',
 'gamesPlayed',
 'fantasyPoints',
 'specialTeams']

In [12]:
import pymannkendall as mk

def convert_to_minutes_seconds(seconds):
    minutes = seconds // 60
    remaining_seconds = seconds % 60
    return f"{minutes}:{remaining_seconds:02d}"


test_data = defense_single.loc[defense_single['name'] == 'Tony DeAngelo'].sort_values('gameDate', ascending=True)
test_data = test_data['toi'].to_list()
result = mk.original_test(test_data)
print(f"{d}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
print(', '.join(formatted_times))

ZeroDivisionError: float division by zero

In [13]:
d_cull = defense_single['name'].value_counts()
d_cull = d_cull[d_cull == 1]
d_cull = d_cull.index.to_list()
d_list = defense_single['name'].unique()
d_list = d_list.tolist()
d_list = [name for name in d_list if name not in d_cull]

In [14]:

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['toi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))

print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['fantasyPoints'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s fantasy points are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        
print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['powerPlayToi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        
print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['shots'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s shots are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num}' for num in test_data]
        print(', '.join(formatted_numbers))

B. Byram's ice time is increasing and the Tau score is 0.29
20:29, 18:57, 17:24, 17:37, 14:03, 19:46, 21:22, 19:39, 18:43, 21:08, 18:28, 18:43, 21:07, 23:01, 23:32, 20:14, 23:13, 22:46, 20:35, 19:19, 19:40, 21:58, 25:15, 22:04, 14:52, 20:46, 20:22, 22:21
O. Ekman-Larsson's ice time is decreasing and the Tau score is -0.61
26:32, 26:16, 25:19, 23:56, 22:04, 22:37, 23:53, 20:22, 26:24, 23:56, 22:21, 23:53, 21:10, 20:37, 23:48, 28:22, 17:59, 18:50, 16:13, 17:47, 13:02, 14:15, 17:13, 17:18, 15:04, 16:16, 17:58
A. Romanov's ice time is increasing and the Tau score is 0.4
22:04, 17:48, 22:06, 21:52, 17:09, 20:35, 23:08, 21:35, 25:16, 25:23, 18:54, 23:37, 21:51, 23:07, 19:33, 18:40, 22:52, 22:04, 30:02, 22:52, 23:43, 24:30, 25:44, 24:41, 25:59, 23:24, 24:17
A. Carrier's ice time is decreasing and the Tau score is -0.44
20:39, 20:50, 19:22, 18:05, 22:37, 21:49, 20:06, 18:38, 19:34, 17:33, 19:20, 20:20, 17:39, 17:33, 15:32, 17:32, 20:53, 19:24, 18:16, 16:24, 15:11, 1:39, 16:00, 20:16
G. Forslin

A. Carrier's fantasy points are decreasing and the Tau score is -0.32
A. Pelech's fantasy points are decreasing and the Tau score is -0.4
Z. Whitecloud's fantasy points are increasing and the Tau score is 0.47
P. Mintyukov's fantasy points are decreasing and the Tau score is -0.28
D. Kulikov's fantasy points are decreasing and the Tau score is -0.29
J. McCabe's fantasy points are increasing and the Tau score is 0.37
B. Pesce's fantasy points are decreasing and the Tau score is -0.47
E. Zamula's fantasy points are decreasing and the Tau score is -0.33
J. Klingberg's fantasy points are decreasing and the Tau score is -0.52
M. Staal's fantasy points are decreasing and the Tau score is -0.49
S. Bolduc's fantasy points are increasing and the Tau score is 0.49
Z. Bogosian's fantasy points are increasing and the Tau score is 0.44
--------------------
J. Trouba's power-play ice time is increasing and the Tau score is 0.28
0:00, 0:06, 0:00, 0:01, 0:00, 0:00, 0:05, 0:29, 0:00, 0:07, 0:16, 0:00, 

In [15]:
f_cull = forwards_single['name'].value_counts()
f_cull = f_cull[f_cull == 1]
f_cull = f_cull.index.to_list()
f_list = forwards_single['name'].unique()
f_list = f_list.tolist()
f_list = [name for name in f_list if name not in f_cull]

In [16]:
#Print a list of forward trends
    
for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['toi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))

print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['fantasyPoints'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s fantasy points are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num:.1f}' for num in test_data]
        print(', '.join(formatted_numbers))
        
print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['powerPlayToi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        
print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['shots'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s shots are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num}' for num in test_data]
        print(', '.join(formatted_numbers))

C. Coyle's ice time is increasing and the Tau score is 0.36
15:56, 17:44, 16:39, 17:01, 15:28, 15:12, 19:32, 14:13, 19:22, 19:30, 15:44, 17:29, 17:18, 17:09, 18:35, 18:05, 18:45, 17:30, 17:56, 15:42, 17:55, 17:16, 19:45, 18:48, 20:03, 20:17
B. Jenner's ice time is decreasing and the Tau score is -0.27
21:01, 17:48, 19:09, 20:24, 22:46, 25:20, 22:07, 22:25, 19:10, 18:19, 18:08, 19:55, 17:27, 17:52, 17:45, 20:43, 17:51, 20:16, 19:36, 15:30, 19:15, 17:40, 19:37, 17:38, 23:24, 20:01, 17:56, 18:14, 12:16
R. Hartman's ice time is decreasing and the Tau score is -0.33
17:56, 17:32, 18:38, 16:27, 16:23, 17:28, 17:28, 17:40, 21:38, 16:05, 16:25, 15:21, 14:46, 15:22, 13:02, 17:54, 18:14, 13:13, 14:02, 12:43, 12:42, 18:51, 16:42
P. Buchnevich's ice time is increasing and the Tau score is 0.46
21:21, 5:27, 17:25, 18:42, 16:45, 20:16, 17:40, 18:14, 17:31, 22:02, 18:30, 16:49, 20:56, 19:38, 22:50, 20:12, 18:38, 18:06, 23:28, 20:21, 19:54, 20:58, 24:03, 23:28, 23:40
S. Reinhart's ice time is decreasi

A. Toropchenko's ice time is decreasing and the Tau score is -0.36
11:47, 15:28, 15:09, 12:03, 12:44, 11:39, 11:41, 10:41, 8:29, 15:13, 10:14, 11:18, 10:55, 12:23, 10:04, 12:50, 13:35, 14:01, 14:04, 14:49, 9:43, 9:23, 9:36, 8:38, 9:19, 8:27, 9:33
J. Eberle's ice time is increasing and the Tau score is 0.3
14:58, 16:43, 17:06, 16:03, 15:33, 19:41, 19:26, 18:32, 17:01, 15:52, 14:58, 21:02, 18:25, 17:22, 16:46, 16:13, 17:02, 15:47, 18:26, 19:15, 20:33, 20:23, 17:15, 19:01, 19:10, 19:20
L. Raymond's ice time is increasing and the Tau score is 0.4
14:55, 17:04, 13:45, 15:31, 15:59, 16:05, 18:52, 17:44, 15:51, 16:33, 16:27, 20:04, 18:40, 18:16, 14:39, 18:39, 18:58, 18:43, 16:35, 17:56, 20:55, 17:37, 19:35, 18:02, 16:45, 20:42, 18:06
J. Gaudreau's ice time is decreasing and the Tau score is -0.31
21:56, 18:13, 17:55, 21:22, 20:57, 24:13, 20:54, 21:32, 19:11, 19:03, 11:55, 19:58, 18:33, 18:35, 16:58, 19:20, 14:42, 19:44, 24:00, 16:55, 20:51, 18:56, 19:09, 21:23, 18:57, 17:46, 16:37, 17:43, 18:

Z. MacEwen's ice time is decreasing and the Tau score is -0.64
6:45, 6:32, 5:02, 6:18, 3:30, 3:59, 4:52, 3:23, 2:38, 4:34
R. Kupari's ice time is decreasing and the Tau score is -0.62
11:17, 9:55, 15:11, 11:20, 11:56, 9:31, 8:49, 8:56, 6:24, 7:42, 7:25, 10:57, 8:34, 6:09, 1:18
T. Nosek's ice time is decreasing and the Tau score is -0.87
12:21, 11:59, 10:34, 8:56, 9:25, 2:55
--------------------
C. Caufield's fantasy points are decreasing and the Tau score is -0.27
2.8, 2.5, 1.8, 4.1, 0.1, 2.2, 5.8, 0.5, 1.7, 0.4, 1.7, 1.8, 3.1, 1.7, 0.2, 0.8, 1.2, 0.4, 0.7, 3.7, 0.3, 2.3, 1.0, 0.4, 1.9, 0.8, 0.9, 0.3
T. Foerster's fantasy points are increasing and the Tau score is 0.28
1.7, 0.8, 0.6, 0.4, 0.2, 1.2, 0.6, 0.9, 0.7, 0.3, 0.5, 0.9, 1.8, 0.7, 1.2, 2.6, 0.0, 0.1, 0.3, 1.4, 2.1, 4.9, 4.2, 2.4, 1.1, 1.9
M. Granlund's fantasy points are increasing and the Tau score is 0.51
0.1, 0.8, 0.8, 0.2, 2.1, 0.6, 0.6, 0.5, 2.6, 1.1, 2.1, 1.0, 0.7, 4.0, 1.3, 0.6, 4.7, 2.7, 3.4, 4.6, 3.1
R. Strome's fantasy

C. Atkinson's power-play ice time is decreasing and the Tau score is -0.27
3:06, 5:10, 4:00, 5:00, 0:59, 1:12, 1:35, 3:25, 4:02, 2:47, 1:45, 5:35, 3:48, 3:32, 3:27, 0:44, 5:12, 1:40, 0:53, 7:00, 1:43, 1:10, 3:09, 2:51, 0:00, 0:00, 0:51
P. Dubois's power-play ice time is decreasing and the Tau score is -0.58
6:17, 5:23, 4:57, 0:00, 5:22, 5:08, 6:13, 4:13, 4:22, 4:58, 3:59, 2:43, 2:51, 3:20, 1:09, 1:59, 3:38, 1:27, 1:59, 2:36, 0:13, 1:42, 0:57, 1:10, 2:01
J. Veleno's power-play ice time is increasing and the Tau score is 0.54
0:00, 0:00, 0:00, 0:00, 0:02, 0:34, 0:00, 0:00, 0:00, 0:00, 0:00, 0:00, 0:00, 0:00, 0:00, 0:41, 0:02, 0:00, 1:26, 2:02, 2:57, 2:14, 1:55, 2:19, 1:40, 4:07, 3:12
W. Foegele's power-play ice time is increasing and the Tau score is 0.36
0:09, 0:00, 0:00, 0:00, 0:17, 0:54, 0:31, 0:00, 0:00, 0:00, 0:00, 0:00, 0:00, 0:00, 0:00, 0:00, 1:04, 1:37, 0:32, 2:33, 0:19, 0:39, 0:38, 1:31, 1:07
F. Zetterlund's power-play ice time is increasing and the Tau score is 0.29
0:00, 0:00,

K. Okposo's shots are increasing and the Tau score is 0.31
2, 0, 1, 0, 3, 0, 0, 0, 1, 0, 3, 0, 1, 2, 2, 4, 1, 0, 1, 5, 2, 3, 5, 1, 3, 0, 2, 6, 2
J. Vesey's shots are increasing and the Tau score is 0.33
2, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 3, 2, 0, 1, 4, 1, 3, 1, 2, 3, 3, 1
J. Schwartz's shots are decreasing and the Tau score is -0.28
1, 3, 2, 2, 3, 3, 5, 3, 2, 6, 3, 1, 2, 3, 2, 3, 1, 3, 0, 1, 1, 1, 1
A. Lafrenière's shots are increasing and the Tau score is 0.34
3, 1, 0, 0, 1, 2, 3, 1, 1, 2, 2, 2, 4, 2, 3, 3, 6, 0, 4, 3, 4, 2, 6, 4, 3, 0
A. Copp's shots are decreasing and the Tau score is -0.39
5, 3, 5, 4, 3, 3, 1, 2, 2, 2, 0, 3, 1, 1, 3, 0, 0, 1, 1, 0, 3, 0, 2, 0, 2, 1, 2
E. Kuznetsov's shots are decreasing and the Tau score is -0.29
2, 2, 2, 2, 3, 2, 5, 4, 4, 1, 3, 3, 1, 3, 2, 2, 0, 1, 0, 2, 0, 2
R. McLeod's shots are increasing and the Tau score is 0.26
1, 1, 0, 0, 1, 1, 1, 0, 3, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 3, 1, 3, 1, 1
G. Smith's shots are decreasing and the Tau score is -0.4

In [51]:
tot_time = pd.DataFrame()

for index, row in team_names_only.iterrows():

    hurricanes = all_df.loc[all_df['team'] == row['abbreviation']].sort_values('gameDate')
    hurricanes_pp = hurricanes.groupby('gameDate').sum()
    
    time = hurricanes_pp['powerPlayToi'].sum()

    formatted_times = [convert_to_minutes_seconds(seconds) for seconds in hurricanes_pp['powerPlayToi']]

#     print(', '.join(formatted_times))
#     print(f"{row['name']} {time}")
    
    temp = pd.DataFrame({'name': [row['name']], 'time': [time]})

    tot_time = pd.concat([tot_time, temp])
tot_time = tot_time.sort_values('time')

tot_time = tot_time.reset_index()
tot_time
# tot_time = tot_time.reset_index()

tot_time['minutes'], tot_time['seconds'] = divmod(tot_time['time'], 60)
tot_time['minutes'] = tot_time['minutes'].astype(str).str.zfill(2)
tot_time['seconds'] = tot_time['seconds'].astype(str).str.zfill(2)
tot_time['total'] = tot_time['minutes'] + ':' + tot_time['seconds']

tot_time

,index,name,time,minutes,seconds,total
0,0,New York Islanders,32287,538,07,538:07
1,0,Toronto Maple Leafs,34660,577,40,577:40
2,0,Dallas Stars,35408,590,08,590:08
3,0,Edmonton Oilers,36474,607,54,607:54
4,0,New York Rangers,36550,609,10,609:10
5,0,Pittsburgh Penguins,36773,612,53,612:53
6,0,San Jose Sharks,37280,621,20,621:20
7,0,Boston Bruins,37926,632,06,632:06
8,0,New Jersey Devils,38263,637,43,637:43
9,0,Washington Capitals,38663,644,23,644:23


In [22]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [23]:
hurricanes = all_df.loc[all_df['team'] == 'CAR'].sort_values('gameDate')
summary_statistics(hurricanes)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
13,8478427,S. Aho,CAR,C,17034,14,5,10,7,43,2,4,2733,1601,-1,30.0,C,F,6.34,2.14
8,8476958,J. Slavin,CAR,D,21547,17,3,7,2,46,7,29,93,3137,8,33.8,D,D,5.65,1.99
19,8482093,S. Jarvis,CAR,C,19780,17,7,6,7,40,24,6,3604,1791,-3,32.9,C,F,5.99,1.94
5,8476869,B. Skjei,CAR,D,21467,17,2,10,2,34,9,23,208,2378,6,30.8,D,D,5.17,1.81
9,8477488,B. Pesce,CAR,D,10792,9,1,1,0,12,6,23,130,1348,3,16.3,D,D,5.44,1.81
6,8476882,T. Teravainen,CAR,L,16717,17,9,3,6,37,4,5,3117,1823,-2,30.6,W,F,6.59,1.80
16,8480829,J. Kotkaniemi,CAR,C,15590,17,6,7,3,38,14,7,2250,125,-2,29.2,C,F,6.74,1.72
15,8480039,M. Necas,CAR,C,18517,17,5,7,4,41,16,6,3485,84,-9,27.7,C,F,5.39,1.63
0,8470613,B. Burns,CAR,D,22286,17,4,4,3,41,4,18,2445,2530,5,27.0,D,D,4.36,1.59
12,8478047,M. Bunting,CAR,L,15252,16,3,6,4,25,5,5,3084,16,-4,19.5,W,F,4.60,1.22


In [56]:
last_season.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['timeOnIce', 'assists', 'goals', 'shots', 'hits', 'powerPlayGoals',
       'powerPlayAssists', 'penaltyMinutes', 'faceOffWins', 'faceoffTaken',
       'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists',
       'blocked', 'plusMinus', 'evenTimeOnIce', 'powerPlayTimeOnIce',
       'shortHandedTimeOnIce', 'position', 'date', 'name', 'team', 'opponent',
       'birthdate', 'id', 'aged', 'faceOffPct', 'pim', 'saves',
       'powerPlaySaves', 'shortHandedSaves', 'evenSaves',
       'shortHandedShotsAgainst', 'evenShotsAgainst', 'powerPlayShotsAgainst',
       'decision', 'savePercentage', 'powerPlaySavePercentage',
       'evenStrengthSavePercentage', 'shutout', 'shortHandedSavePercentage',
       'gamePlayed', 'fantasyPoints', 'specialTeams', 'shots_against',
       'goals_against', 'decisionPoints', 'wins'],
      dtype='object')>

In [66]:
last_season = pd.read_csv('data/2022-23.csv')
player = last_season.loc[(last_season['name'] == 'Alex Ovechkin') & (last_season['date'] < '2022-12-03')]
player
player.groupby('name').sum()

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,...,evenStrengthSavePercentage,shutout,shortHandedSavePercentage,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
name,,,,,,,,,,,,,,,,,,,,,
Alex Ovechkin,30977,12,13,103,69,6,6,16,1,4,...,0.0,0,0.0,25.0,64.2,12.0,0.0,0.0,0.0,0.0
